In [32]:
import pandas as pd
import numpy as np
import requests

In [3]:
barts_hotspots = pd.read_csv('data/barts_hotspots.csv')
barts_to_all = pd.read_csv('data/barts_to_all.csv')
hotspots_to_all = pd.read_csv('data/hotspots_to_all.csv')
hours_q1 = pd.read_csv('data/hours_q1.csv')
hours_q2 = pd.read_csv('data/hours_q2.csv')

In [73]:
barts_hotspots.to_csv('barts_hotspots_updated.csv')

In [50]:
barts_hotspots.columns = ['Date',
 'Origin ID',
 'Origin Name',
 'Destination ID',
 'Destination Name',
 'Daily Mean Travel Time',
 'Daily Range - Lower',
 'Daily Range - Upper',
 'AM Mean Travel Time',
 'AM Range - Lower',
 'AM Range - Upper',
 'PM Mean Travel Time',
 'PM Range - Lower Bound Travel Time',
 'PM Range - Upper Bound Travel Time',
 'Midday Mean Travel Time',
 'Midday Range - Lower',
 'Midday Range - Upper',
 'Evening Mean Travel Time',
 'Evening Range - Lower',
 'Evening Range - Upper',
 'Early Morning Mean Travel Time',
 'Early Morning Range - Lower',
 'Early Morning Range - Upper',
 'weekday']

In [52]:
barts_hotspots.sort_values('Date')

,Date,Origin ID,Origin Name,Destination ID,Destination Name,Daily Mean Travel Time,Daily Range - Lower,Daily Range - Upper,AM Mean Travel Time,AM Range - Lower,...,Midday Mean Travel Time,Midday Range - Lower,Midday Range - Upper,Evening Mean Travel Time,Evening Range - Lower,Evening Range - Upper,Early Morning Mean Travel Time,Early Morning Range - Lower,Early Morning Range - Upper,weekday
2106,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",684.0,381.0,1228.0,NaN,NaN,...,432.0,338.0,551.0,NaN,NaN,NaN,851.0,478.0,1515.0,1
2524,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3692,2nd Street and Stevenson Street (Montgomery BA...,483.0,319.0,729.0,NaN,NaN,...,464.0,343.0,627.0,427.0,322.0,565.0,493.0,312.0,777.0,1
2091,2019-01-01,3603,"Embarcadero, San Francisco, CA",3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",1315.0,1074.0,1609.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2891,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3760,"Powell BART Station, Market St and Powell St, ...",677.0,453.0,1012.0,NaN,NaN,...,572.0,426.0,769.0,NaN,NaN,NaN,730.0,467.0,1139.0,1
2642,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3692,2nd Street and Stevenson Street (Montgomery BA...,1559.0,1233.0,1970.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1809.0,1551.0,2110.0,1
3005,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3760,"Powell BART Station, Market St and Powell St, ...",1576.0,1279.0,1942.0,NaN,NaN,...,1545.0,1248.0,1912.0,NaN,NaN,NaN,1685.0,1427.0,1989.0,1
2531,2019-01-01,3692,2nd Street and Stevenson Street (Montgomery BA...,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",517.0,274.0,973.0,NaN,NaN,...,396.0,248.0,630.0,311.0,184.0,525.0,571.0,297.0,1096.0,1
2672,2019-01-01,3394,"Fisherman's Wharf, San Francisco, CA",3692,2nd Street and Stevenson Street (Montgomery BA...,1125.0,756.0,1674.0,NaN,NaN,...,943.0,717.0,1240.0,761.0,631.0,918.0,1736.0,1011.0,2983.0,1
2078,2019-01-01,3394,"Fisherman's Wharf, San Francisco, CA",3603,"Embarcadero, San Francisco, CA",1124.0,670.0,1886.0,NaN,NaN,...,869.0,618.0,1221.0,643.0,508.0,814.0,1516.0,851.0,2699.0,1
1864,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3603,"Embarcadero, San Francisco, CA",1450.0,1115.0,1886.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [64]:
def stats(df):
    stats = df.describe().T
    stats['filled'] = stats['count']/len(df)
    return stats[['filled']]

In [57]:
barts_hotspots_filled = barts_hotspots.copy()

In [78]:
def conditional_fill(df,col):
    df_filled = df.copy()
    df_filled[col] = barts_hotspots_filled.groupby(['Origin ID','weekday','Destination ID'])[col].transform(lambda x: x.fillna(x.median()))
    return df_filled

In [ ]:
barts_hotspots_filled = conditional_fill(barts_hotspots_filled,'Early Morning Range - Lower')

In [77]:
stats(barts_hotspots_filled)

,filled
Origin ID,1.000000
Destination ID,1.000000
Daily Mean Travel Time,0.982198
Daily Range - Lower,0.982198
Daily Range - Upper,0.982198
AM Mean Travel Time,0.550031
AM Range - Lower,0.550031
AM Range - Upper,0.550031
PM Mean Travel Time,0.715163
PM Range - Lower Bound Travel Time,0.715163


In [66]:
stats(barts_hotspots)

,filled
Origin ID,1.000000
Destination ID,1.000000
Daily Mean Travel Time,0.982198
Daily Range - Lower,0.982198
Daily Range - Upper,0.982198
AM Mean Travel Time,0.550031
AM Range - Lower,0.550031
AM Range - Upper,0.550031
PM Mean Travel Time,0.715163
PM Range - Lower Bound Travel Time,0.715163


In [79]:
barts_hotspots

,Date,Origin ID,Origin Name,Destination ID,Destination Name,Daily Mean Travel Time,Daily Range - Lower,Daily Range - Upper,AM Mean Travel Time,AM Range - Lower,...,Midday Mean Travel Time,Midday Range - Lower,Midday Range - Upper,Evening Mean Travel Time,Evening Range - Lower,Evening Range - Upper,Early Morning Mean Travel Time,Early Morning Range - Lower,Early Morning Range - Upper,weekday
0,2019-06-09,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3603,"Embarcadero, San Francisco, CA",1588.0,1054.0,2392.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
1,2019-04-28,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",639.0,512.0,796.0,NaN,NaN,...,654.0,524.0,817.0,608.0,492.0,753.0,NaN,NaN,NaN,6
2,2019-05-18,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",730.0,520.0,1024.0,NaN,NaN,...,791.0,590.0,1061.0,569.0,406.0,797.0,NaN,NaN,NaN,5
3,2019-04-18,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",512.0,371.0,705.0,648.0,461.0,...,528.0,398.0,702.0,442.0,321.0,608.0,363.0,297.0,443.0,3
4,2019-04-08,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",505.0,364.0,701.0,621.0,489.0,...,429.0,339.0,542.0,502.0,354.0,711.0,NaN,NaN,NaN,0
5,2019-06-30,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",616.0,385.0,987.0,NaN,NaN,...,NaN,NaN,NaN,386.0,276.0,541.0,NaN,NaN,NaN,6
6,2019-04-23,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",682.0,500.0,931.0,NaN,NaN,...,756.0,577.0,991.0,550.0,406.0,745.0,NaN,NaN,NaN,1
7,2019-05-13,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",634.0,457.0,878.0,NaN,NaN,...,584.0,470.0,726.0,611.0,424.0,880.0,NaN,NaN,NaN,0
8,2019-05-15,3394,"Fisherman's Wharf, San Francisco, CA",3603,"Embarcadero, San Francisco, CA",878.0,621.0,1243.0,904.0,650.0,...,732.0,586.0,915.0,795.0,612.0,1032.0,568.0,438.0,736.0,2
9,2019-04-14,3603,"Embarcadero, San Francisco, CA",3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",996.0,853.0,1164.0,NaN,NaN,...,1063.0,928.0,1219.0,NaN,NaN,NaN,NaN,NaN,NaN,6
